In [ ]:
# 自动下载一些必要库
!pip install torch torchvision
!pip install numpy matplotlib seaborn pillow

In [ ]:
import torch
import torchvision
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
# from tensorboardX import SummaryWriter  # 本code的tensorboard可视化方法都被注释掉了


### 下载数据集 && 数据预处理
这段code将CIFAR10的数据集下载在本地的`./data`文件夹下
在这里我们使用了transforms模块对数据进行预处理，

意义：对数据进行预处理，可以提高模型的泛化能力，同时也可以加快模型的训练速度，提高模型的精度。

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# CIFAR-10 dataset 导入
train_dataset = torchvision.datasets.CIFAR10(root='./data/',
                                             train=True, 
                                             transform=transform,
                                             download=True)

test_dataset = torchvision.datasets.CIFAR10(root='./data/',
                                            train=False, 
                                            transform=transforms.ToTensor())

### 自定义CIFAR-10 数据集类
用于读取数据集--CIFAR-10,并且对数据集进行预处理

In [ ]:
class CIFAR10Dataset(Dataset):
    def __init__(self, root_dir, train=True, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.train = train

        # 根据训练或测试集加载不同的文件
        self.data = []
        self.labels = []
        if self.train:
            for i in range(1, 6):
                file = os.path.join(self.root_dir, 'data_batch_' + str(i))
                with open(file, 'rb') as fo:
                    dict = pickle.load(fo, encoding='bytes')
                    self.data.append(dict[b'data'])
                    self.labels += dict[b'labels']
            self.data = torch.cat([torch.tensor(d).view(-1, 3, 32, 32) for d in self.data])
        else:
            file = os.path.join(self.root_dir, 'test_batch')
            with open(file, 'rb') as fo:
                dict = pickle.load(fo, encoding='bytes')
                self.data = torch.tensor(dict[b'data']).view(-1, 3, 32, 32)
                self.labels = dict[b'labels']

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image = self.data[idx]
        label = self.labels[idx]
        
        # 将图像数据格式从[channels, height, width]转换为PIL图像格式
        image = Image.fromarray(image.numpy().transpose((1, 2, 0)))

        if self.transform:
            image = self.transform(image)

        return image, label

In [ ]:

# 使用torch函数加载数据集
#trainset = datasets.CIFAR10(root='/mnt/ssd/dataset_CIFAR-10', train=True, download=True, transform=transform)
#  使用自定义的CIFAR10Dataset类加载数据集
trainset = CIFAR10Dataset(root_dir='./data/cifar-10-batches-py/', train=True, transform=transform)
trainloader = DataLoader(trainset, batch_size=16, shuffle=True, num_workers=4)

# 加载测试数据集
#testset = datasets.CIFAR10(root='/mnt/ssd/dataset_CIFAR-10/cifar-10-batches-py', train=False, download=True, transform=transform)
# 使用自定义的CIFAR10Dataset类加载数据集
testset = CIFAR10Dataset(root_dir='./data/cifar-10-batches-py/', train=False, transform=transform)
testloader = DataLoader(testset, batch_size=16, shuffle=False, num_workers=4)

# CIFAR-10类别
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')


#### 加载数据集后(并且完成预处理了), 
#### 可视化一些训练样本(dataloader中的数据)

In [ ]:
def imshow(img):
    img = img / 2 + 0.5     # 反标准化normalize
    npimg = img.numpy()     # 将Tensor转换为ndarray
    plt.imshow(np.transpose(npimg, (1, 2, 0))) # 转换为[height, width, channels]
    plt.show()

# 获取一些随机训练图片
dataiter = iter(trainloader)    # 将trainloader转换为迭代器
images, labels = next(dataiter) # 获取一组图像和标签

# 显示批量图片
imshow(torchvision.utils.make_grid(images)) # 将一个batch_size的图像拼接成网格
print(','.join('%5s' % classes[labels[j]] for j in range(16)))

'''
# 显示一张图片与标签
imshow(images[0])
print(classes[labels[0]])
'''

## 简单CNN模型--LeNet

In [ ]:
class LeNet(nn.Module):
    def __init__(self): #定义了网络的结构
        super(LeNet, self).__init__()
        # 第一个卷积层: 输入通道数为3（对于RGB图像），输出通道数为6，卷积核大小为5x5
        self.conv1 = nn.Conv2d(3, 6, 5)
        # 第二个卷积层: 输入通道数为6，输出通道数为16，卷积核大小为5x5
        self.conv2 = nn.Conv2d(6, 16, 5)
        # 最大池化层，使用2x2窗口, 步长为2
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        # 全连接层1: 输入特征数为16 * 6 * 6（16个通道，每个通道的大小是5x5），输出特征数为120
        self.fc1 = nn.Linear(16 * 5 * 5, 120)   # 5*5是通过计算得到的图像大小
        # 全连接层2: 输入特征数为120，输出特征数为84
        self.fc2 = nn.Linear(120, 84)
        # 全连接层3（输出层）: 输入特征数为84，输出特征数为10（对应10个类别）
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):   #定义了数据通过网络的方式
        # 应用第一个卷积层 followed by ReLU激活函数和池化
        x = self.conv1(x)
        x = F.relu(x)
        x = self.pool(x)
        # 应用第二个卷积层 followed by ReLU激活函数和池化
        x = self.pool(F.relu(self.conv2(x)))
        # 展平所有除批次维度以外的维度
        x = x.view(-1, 16 * 5 * 5)
        # 应用第一个全连接层和ReLU激活函数
        x = self.fc1(x)
        x = F.relu(x)
        # 应用第二个全连接层和ReLU激活函数
        x = F.relu(self.fc2(x))
        # 应用输出层
        x = self.fc3(x)
        return x


In [ ]:
# 检查是否有可用的GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 定义模型
#model = ResNet(BasicBlock, [2, 2, 2, 2])
model = LeNet()
model.to(device)

# 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# 开始训练

In [ ]:
#モデルの学習
train_loss_list = []
train_acc_list = []
val_loss_list = []
val_acc_list = []
epoch = 10
# 使用tensorboardX进行可视化
#writer = SummaryWriter(comment='test_comment', filename_suffix="CIFAR10_Lenet")

for i in range(epoch):
    print('-'*5, 'Epoch [{}/{}] start'.format(i, epoch-1), '-'*5)
    epoch_loss = 0
    epoch_accuracy = 0
    model.train()
    for image, target in tqdm(trainloader):
        image, target = image.to(device), target.to(device)
        output = model(image).squeeze()
        loss = criterion(output, target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        acc = (output.argmax(dim=1) == target).float().mean()
        epoch_accuracy += acc / len(trainloader)
        epoch_loss += loss / len(trainloader)
        
    # モデルの検証、評価モード切替
    model.eval()

    # 検証データのデータローダから読み込み尽くすまでループ
    with torch.no_grad():
        epoch_val_accuracy=0
        epoch_val_loss = 0

        for image, target in testloader:

            # 入力、正解を GPU へ移動
            image, target = image.cuda(), target.cuda()

            # モデルに入力を順伝播させ予測を出力
            output = model(image).squeeze()

            # 損失関数で予測と正解の誤差や精度を導出
            loss = criterion(output, target)

            acc = (output.argmax(dim=1) == target).float().mean()
            epoch_val_accuracy += acc / len(testloader)
            epoch_val_loss += loss / len(testloader)

    print('epoch',epoch)
    print('acc',epoch_accuracy)
    print('loss',epoch_loss)
    print('val_acc',epoch_val_accuracy)
    print('val_loss',epoch_val_loss)
    train_loss_list.append(epoch_loss)
    train_acc_list.append(epoch_accuracy)
    val_loss_list.append(epoch_val_loss)
    val_acc_list.append(epoch_val_accuracy)
#     writer.add_scalar('train_loss', epoch_loss, i)
#     writer.add_scalar('train_acc', epoch_accuracy, i)
#     writer.add_scalar('val_loss', epoch_val_loss, i)
#     writer.add_scalar('val_acc', epoch_val_accuracy, i)

# writer.close()

#### 结果可视化
1. 训练集和验证集的accuracy曲线
2. 训练集和验证集的loss曲线

In [ ]:
#学習曲線の描画
train_acc = []
train_loss = []
val_acc = []
val_loss = []

for i in range(epoch):
  train_acc2 = train_acc_list[i].cpu()
  train_acc3 = train_acc2.clone().numpy()
  train_acc.append(train_acc3)

  train_loss2 = train_loss_list[i].cpu()
  train_loss3 = train_loss2.detach().numpy()
  train_loss.append(train_loss3)

  val_acc2 = val_acc_list[i].cpu()
  val_acc3 = val_acc2.clone().numpy()
  val_acc.append(val_acc3)

  val_loss2 = val_loss_list[i].cpu()
  val_loss3 = val_loss2.clone().numpy()
  val_loss.append(val_loss3)


#取得したデータをグラフ化する
sns.set()
num_epochs = epoch
fig = plt.subplots(figsize=(12,4), dpi=80)

ax1 = plt.subplot(1,2,1)
ax1.plot(range(num_epochs), train_acc, c='b', label='train acc')
ax1.plot(range(num_epochs), val_acc, c='r', label='val acc')
ax1.set_xlabel('epoch', fontsize='12')
ax1.set_ylabel('accuracy', fontsize='12')
ax1.set_title('training and val acc', fontsize='14')
ax1.legend(fontsize='12')

ax2 = plt.subplot(1,2,2)
ax2.plot(range(num_epochs), train_loss, c='b', label='train loss')
ax2.plot(range(num_epochs), val_loss, c='r', label='val loss')
ax2.set_xlabel('epoch', fontsize='12')
ax2.set_ylabel('loss', fontsize='12')
ax2.set_title('training and val loss', fontsize='14')
ax2.legend(fontsize='12')